In this notebook, I attempt to use Recurrent Neural Networks (RNNs), Gated Recurrent Units (GRUs) and Long-Short Term Memory (LSTM) to conduct classification

In [10]:
import os
import torch
import numpy as np
import pandas as pd

from torch import nn
from torch.utils.data import Dataset, DataLoader
# from torchvision import datasets, transforms

Reading in the data

In [3]:
train_df = pd.read_csv('data/DL/DL_train.csv')
train_df.head()

,text,humor
0,watch this swimmer disappear into winter storm...,False
1,"they laughed at reagan, too: trump's ideas wil...",False
2,"hey, are you cold? go over to the corner, it i...",True
3,cannot get a standing desk? these are almost a...,False
4,want to hear a joke about my penis? never mind...,True


In [4]:
test_df = pd.read_csv('data/DL/DL_test.csv')
test_df.head()

,text,humor
0,thought up a reddit joke today. when is a tria...,True
1,how much do pirates pay for corn? a buck an ear!,True
2,hillary clinton sent her book to every gop can...,False
3,italian unions lambast new museum boss for wor...,False
4,life below the ocean’s surface wholly depends ...,False


Setting up the hyperparameters

In [6]:
INPUT_SIZE = 784
HIDDEN_SIZE = 300
NUM_CLASSES = 2
NUM_EPOCHS = 2
BATCH_SIZE = 64
LEARNING_RATE = 0.001

Defining classes for the RNN and dataset

In [5]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()

        self.hidden_size = hidden_size
        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.softmax = nn.LogSoftMax(dim=1)
    
    def forward(self, input_tensor, hidden_tensor):
        combined = torch.cat((input_tensor, hidden_tensor), 1)

        hidden = self.i2h(combined)
        output = self.i2o(combined)
        output = self.softmax(output)
        return output, hidden

    def init_hidden(self):
        return torch.zeros(1, self.hidden_size)

Create our custom dataset

The custom dataset should inherit Dataset and override the following methods:
1. \_\_len\_\_ so that len(dataset) returns the size of the dataset.
2. \_\_getitem\_\_ to support the indexing such that dataset[i] can be used to get the ith sample.

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, df):
        self.df = df

    def __getitem__(self, index):
        row = self.df.iloc[index].to_numpy()
        features = row[1:]
        label = row[0]
        return features, label

    def __len__(self):
        return len(self.df)

In [7]:
train_dataloader = DataLoader(train_df, batch_size=BATCH_SIZE)
test_dataloader = DataLoader(test_df, batch_size=BATCH_SIZE)

In [9]:
for X, y in test_dataloader:
    print(f"Text: {X}")
    print(f"Value: {y}")
    break

KeyError: 0